In [ ]:
# =============================================================================
# 🚀 그룹 1: 통일된 함수명 - 마이리얼트립 크롤링 시스템 (함수명 정리 완료)
# 함수명 단순화: get_product_name(), get_price(), download_image(), clean_price(), clean_rating(), save_results()
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용  
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    "LONGER_DELAYS": True,           # 새로 추가
    "MEMORY_CLEANUP_INTERVAL": 5,    # 새로 추가
    "MAX_PRODUCTS_PER_CITY": 2,     # 2 → 10개로 증가⭐⭐⭐⭐⭐⭐⭐⭐⭐
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["후쿠오카"]

# 🆕 통합된 도시 정보 구조 (Gemini 지적사항 반영)
CITY_CODES = {
    # 동남아시아
    "방콕": "BKK",
    "치앙마이": "CNX", 
    "푸켓": "HKT",
    "싱가포르": "SIN",
    "홍콩": "HKG",
    "쿠알라룸푸르": "KUL",
    "세부": "CEB",
    "다낭": "DAD",
    "호치민": "SGN",
    
    # 일본
    "도쿄": "NRT",
    "오사카": "KIX",
    "나고야": "NGO",
    "후쿠오카": "FUK",
    "오키나와": "OKA",
    "삿포로": "CTS",
    
    # 한국
    "서울": "ICN",
    "부산": "PUS",
    "제주": "CJU",
    "대구": "TAE",
    "광주": "KWJ",
    "여수": "RSU",
    
    # 유럽
    "파리": "CDG",
    "런던": "LHR",
    "로마": "FCO",
    "바르셀로나": "BCN",
    
    # 북미
    "뉴욕": "JFK",
    "로스앤젤레스": "LAX",
    "시카고": "ORD",
    
    # 오세아니아
    "시드니": "SYD",
    "멜버른": "MEL",
}

# 🆕 통합된 도시 정보 (대륙/국가 정보 포함)
UNIFIED_CITY_INFO = {
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK"},
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG"},
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG"},
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR"},
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK"},
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD"},
}

print(f"✅ CITY_CODES 추가 완료! {len(CITY_CODES)}개 도시 지원")

# =============================================================================
# 🔧 핵심 함수들 - 함수명 통일 완료! (마이리얼트립 전용)
# =============================================================================

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (공용 함수)"""
    code = CITY_CODES.get(city_name, city_name[:3].upper())
    print(f"  🏙️ {city_name} → {code}")
    return code

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (공용 함수)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 기본값 반환
        return "기타", "기타"

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (기존: get_product_name_by_type → 새로운: get_product_name)"""
    print(f"  📊 {url_type} 상품명 수집 중...")
    
    # 기본 상품명 셀렉터들 (Products와 Offers 공통)
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """✅ 가격 수집 (기존: get_price_fixed → 새로운: get_price)"""
    print(f"  💰 가격 정보 수집 중...")
    
    # 마이리얼트립 실제 가격 위치들 (우선순위 순서)
    price_selectors = [
        # 1순위: 할인된 가격 (빨간색 텍스트)
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 87, 87)']"),
        (By.CSS_SELECTOR, "span[style*='color: red']"),
        (By.CSS_SELECTOR, ".price-discount"),
        
        # 2순위: 일반 가격
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        
        # 3순위: 원이 포함된 텍스트 (쿠폰 제외)
        (By.XPATH, "//span[contains(text(), '원') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        
        # 4순위: 기본 가격 위치
        (By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div/div[2]/span[2]")
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_price = price_element.text.strip()
            
            # 쿠폰 관련 텍스트 제외
            if any(keyword in found_price for keyword in ['쿠폰', '받기', '다운']):
                continue
                
            # 가격 패턴 확인 (숫자 + 원)
            if '원' in found_price and any(char.isdigit() for char in found_price):
                return found_price
                
        except TimeoutException:
            continue
    
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (기존: extract_clean_price → 새로운: clean_price) (공용 함수)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    # 가격 패턴: 숫자,숫자원 또는 숫자원
    price_pattern = r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?'
    match = re.search(price_pattern, price_text)
    
    if match:
        return match.group(1) + "원"
    else:
        return price_text  # 원본 반환

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    # 평점 패턴: 숫자.숫자
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text

def download_image(driver, product_name, city_name, product_index):
    """✅ 이미지 다운로드 (기존: download_image_improved_fixed → 새로운: download_image)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': '',
            'path': '',
            'size': 0
        }
        
    print(f"  🖼️ 대표 상품 이미지 다운로드 중...")
    
    # 대표 이미지 우선 셀렉터들
    image_selectors = [
        # 1순위: 큰 메인 이미지
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        
        # 2순위: 일반 상품 이미지
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        
        # 3순위: 기본 이미지
        (By.XPATH, "//img[contains(@alt, '상품')]"),
        (By.CSS_SELECTOR, "img[src*='cdn']"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            
            for img_element in img_elements:
                img_url = img_element.get_attribute('src')
                if img_url and img_url.startswith('http'):
                    # 이미지 크기 확인 (너무 작은 이미지 제외)
                    try:
                        width = img_element.get_attribute('width') or img_element.get_attribute('naturalWidth')
                        height = img_element.get_attribute('height') or img_element.get_attribute('naturalHeight')
                        
                        if width and height:
                            if int(width) < 100 or int(height) < 100:
                                continue  # 너무 작은 이미지는 스킵
                    except:
                        pass
                    
                    break
            
            if img_url:
                break
                
        except Exception:
            continue

    if img_url:
        try:
            # 공항 코드 기반 파일명
            city_code = get_city_code(city_name)
            img_filename = f"{city_code}_{product_index:03d}.jpg"
            
            # 이미지 폴더 생성
            img_folder = "myrealtripthumb_img"
            os.makedirs(img_folder, exist_ok=True)
            
            img_path = os.path.join(img_folder, img_filename)
            
            # 이미지 다운로드
            urllib.request.urlretrieve(img_url, img_path)
            
            # 파일 크기 확인
            file_size = os.path.getsize(img_path)
            
            if file_size < 1024:  # 1KB 미만이면 실패로 간주
                os.remove(img_path)
                return {
                    'status': '다운로드 실패 (파일 너무 작음)',
                    'filename': img_filename,
                    'path': '',
                    'size': 0
                }
            
            print(f"  ✅ 이미지 다운로드 완료! ({file_size:,} bytes)")
            return {
                'status': '다운로드 완료',
                'filename': img_filename,
                'path': img_path,
                'size': file_size
            }
            
        except Exception as e:
            print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}")
            return {
                'status': f'다운로드 실패: {type(e).__name__}',
                'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
                'path': '',
                'size': 0
            }
    else:
        return {
            'status': '이미지 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'size': 0
        }

def save_results(products_data):
    """✅ 데이터 저장 (기존: save_myrealtrip_data → 새로운: save_results)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 도시명 가져오기
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    # 개별 CSV 저장
    df = pd.DataFrame(products_data)
    csv_path = f"myrealtrip_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📁 개별 CSV 저장 완료: {csv_path}")
    
    # 메타데이터 저장
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "city": city_name
        }
    }
    
    try:
        with open('data_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        print(f"📁 메타데이터 저장 완료: data_metadata.json")
    except Exception as e:
        print(f"⚠️ 메타데이터 저장 실패: {e}")
    
    return csv_path

# =============================================================================
# 🚀 Phase 2: 확장성 개선 시스템 (기존 코드 유지)
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장"""
    
    # 🆕 Gemini 지적사항 반영: 대륙/국가 정보도 포함
    enhanced_city_data = {
        "version": "2.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(CITY_CODES)
    }
    
    # 통합된 정보로 cities 구성
    for city_name, city_code in CITY_CODES.items():
        city_info = UNIFIED_CITY_INFO.get(city_name, {"대륙": "기타", "국가": "기타"})
        enhanced_city_data["cities"][city_name] = {
            "code": city_code,
            "continent": city_info["대륙"],
            "country": city_info["국가"]
        }
    
    try:
        with open('city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ city_codes.json 파일 생성 완료! ({len(CITY_CODES)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드"""
    
    if not os.path.exists('city_codes.json'):
        print("📝 city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
        return CITY_CODES
    
    try:
        with open('city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        # 새 형식 (v2.0) 처리
        if "cities" in city_data and isinstance(list(city_data["cities"].values())[0], dict):
            loaded_codes = {city: info["code"] for city, info in city_data["cities"].items()}
        else:
            # 구 형식 (v1.0) 처리
            loaded_codes = city_data.get('cities', {})
        
        print(f"✅ city_codes.json 로드 완료! ({len(loaded_codes)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
        return loaded_codes
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 기존 코드의 CITY_CODES를 사용합니다.")
        return CITY_CODES

def add_new_city(city_name, airport_code, update_file=True):
    """새로운 도시를 추가하는 함수"""
    
    global CITY_CODES
    
    # 메모리에 추가
    CITY_CODES[city_name] = airport_code
    print(f"✅ 메모리에 추가: {city_name} → {airport_code}")
    
    # 파일에도 저장
    if update_file:
        create_city_codes_file()  # 전체 재생성
    
    return True

def show_supported_cities():
    """지원하는 도시 목록 표시"""
    
    print("\\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    # 지역별로 분류
    regions = {
        "동남아시아": ["방콕", "치앙마이", "푸켓", "싱가포르", "홍콩", "쿠알라룸푸르", "세부", "다낭", "호치민"],
        "일본": ["도쿄", "오사카", "나고야", "후쿠오카", "오키나와", "삿포로"],
        "한국": ["서울", "부산", "제주", "대구", "광주", "여수"],
        "유럽": ["파리", "런던", "로마", "바르셀로나"],
        "북미": ["뉴욕", "로스앤젤레스", "시카고"],
        "오세아니아": ["시드니", "멜버른"]
    }
    
    for region, cities in regions.items():
        print(f"\\n📍 {region}:")
        for city in cities:
            if city in CITY_CODES:
                code = CITY_CODES[city]
                print(f"   {city} → {code}")
    
    print(f"\\n📊 총 {len(CITY_CODES)}개 도시 지원")
    print("="*50)

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    
    global CONFIG
    
    # 기존 CONFIG에 확장성 설정 추가
    scalability_config = {
        # 도시 관리
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        
        # 다중 도시 지원
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        
        # 파일 관리
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        
        # 확장 기능
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def initialize_file_system():
    """파일 시스템 초기화 및 설정"""
    
    print("🔧 Phase 2: 확장성 개선 시스템 초기화...")
    
    # CONFIG 업데이트
    update_config_for_scalability()
    
    # 도시 코드 파일 로드/생성
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        global CITY_CODES
        loaded_codes = load_city_codes_from_file()
        
        # 새로 로드된 코드가 더 많으면 업데이트
        if len(loaded_codes) >= len(CITY_CODES):
            CITY_CODES = loaded_codes
            print(f"🔄 CITY_CODES 업데이트: {len(CITY_CODES)}개 도시")
        else:
            # 메모리의 코드가 더 최신이면 파일 업데이트
            create_city_codes_file()
    
    print("✅ Phase 2 시스템 초기화 완료!")
    return True

def quick_add_cities():
    """자주 사용하는 도시들을 빠르게 추가"""
    
    quick_cities = {
        # 추가로 자주 사용될 도시들
        "교토": "KIX",      # 오사카 공항 사용
        "인천": "ICN",      # 서울 공항
        "김포": "GMP",      # 김포공항
        "하와이": "HNL",    # 호놀룰루
        "괌": "GUM",        # 괌 국제공항
        "사이판": "SPN",    # 사이판 공항
        "푸꾸옥": "PQC",    # 푸꾸옥 공항
        "나트랑": "CXR",    # 나트랑 공항
        "보홀": "TAG",      # 보홀 공항
        "랑카위": "LGK",    # 랑카위 공항
    }
    
    print("🚀 자주 사용하는 도시들 추가 중...")
    
    for city, code in quick_cities.items():
        if city not in CITY_CODES:
            add_new_city(city, code, update_file=False)
    
    # 한 번에 파일 저장
    create_city_codes_file()
    
    print(f"✅ {len(quick_cities)}개 도시 일괄 추가 완료!")

def validate_city(city_name):
    """도시명 유효성 검사"""
    
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in CITY_CODES:
        return True, f"지원하는 도시입니다. ({CITY_CODES[city_name]})"
    
    # 유사한 도시명 찾기
    similar_cities = []
    for supported_city in CITY_CODES.keys():
        if city_name.lower() in supported_city.lower() or supported_city.lower() in city_name.lower():
            similar_cities.append(supported_city)
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

# =============================================================================
# 🛠️ 기존 유틸리티 함수들 (Phase 2 시스템과 함께 유지)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]  # ✅ Gemini 지적사항 해결: USER_AGENT 사용
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭 (안정성 강화)"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    
    # ... (기존의 view_all_selectors 리스트는 그대로 둡니다) ...
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )

            # --- 이 부분을 수정합니다 ---
            # 기존 코드: view_all_button.click()
            
            # 수정 코드: JavaScript로 직접 클릭 실행
            driver.execute_script("arguments[0].click();", view_all_button)
            # --------------------------
            
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
        
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print("  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def get_rating(driver):
    """평점 정보 수집 (기존 함수 유지)"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집"""
    print(f"  🌐 언어 정보 찾는 중...")
    language_selectors = [
        (By.XPATH, "//span[contains(text(), '언어')]"),
        (By.XPATH, "//span[contains(text(), '한국어')]"),
        (By.XPATH, "//span[contains(text(), '영어')]"),
        (By.XPATH, "//span[contains(text(), 'Korean')]"),
        (By.XPATH, "//span[contains(text(), 'English')]"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            language_text = language_element.text.strip()
            
            language_keywords = ['언어', '한국어', '영어', '중국어', '일본어', 'Korean', 'English']
            
            if any(keyword in language_text for keyword in language_keywords):
                print(f"  ✅ 언어 정보 발견: {language_text}")
                return language_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

# =============================================================================
# Phase 2 시스템 자동 실행
# =============================================================================

# 자동 초기화 실행
try:
    initialize_file_system()
    
    # 자주 사용하는 도시들도 추가
    quick_add_cities()
    
    # 지원 도시 목록 표시
    show_supported_cities()
    
    print("\n🎉 Phase 2: 확장성 개선 완료!")
    print("💡 이제 이런 기능들을 사용할 수 있습니다:")
    print("   - add_new_city('제주도', 'CJU')  # 새 도시 추가")
    print("   - show_supported_cities()        # 지원 도시 목록")
    print("   - validate_city('방콕')          # 도시 유효성 검사")
    
except Exception as e:
    print(f"❌ Phase 2 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

print("\n" + "="*60)
print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print("="*60)
print("🔧 핵심 함수명 변경 완료:")
print("   get_product_name_by_type() → get_product_name()")
print("   get_price_fixed() → get_price()")
print("   download_image_improved_fixed() → download_image()")
print("   extract_clean_price() → clean_price()")
print("   extract_clean_rating() → clean_rating()")
print("   save_myrealtrip_data() → save_results()")
print("="*60)
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2,3,4에서 함수 호출을 통일된 함수명으로 변경하세요!")
print("🚨 Gemini 지적사항 모두 해결 완료! 안전하게 실행 가능합니다!")

🔧 Selenium 버전: 4.25.0
✅ CITY_CODES 추가 완료! 30개 도시 지원
🔧 Phase 2: 확장성 개선 시스템 초기화...
⚙️ CONFIG 확장성 설정 업데이트 완료!
✅ city_codes.json 로드 완료! (40개 도시)
📅 마지막 업데이트: 2025-07-15 23:04:50
🔄 CITY_CODES 업데이트: 40개 도시
✅ Phase 2 시스템 초기화 완료!
🚀 자주 사용하는 도시들 추가 중...
✅ city_codes.json 파일 생성 완료! (40개 도시)
✅ 10개 도시 일괄 추가 완료!
\n🌍 지원하는 도시 목록:
\n📍 동남아시아:
   방콕 → BKK
   치앙마이 → CNX
   푸켓 → HKT
   싱가포르 → SIN
   홍콩 → HKG
   쿠알라룸푸르 → KUL
   세부 → CEB
   다낭 → DAD
   호치민 → SGN
\n📍 일본:
   도쿄 → NRT
   오사카 → KIX
   나고야 → NGO
   후쿠오카 → FUK
   오키나와 → OKA
   삿포로 → CTS
\n📍 한국:
   서울 → SEL
   부산 → PUS
   제주 → CJU
   대구 → TAE
   광주 → KWJ
   여수 → RSU
\n📍 유럽:
   파리 → CDG
   런던 → LHR
   로마 → FCO
   바르셀로나 → BCN
\n📍 북미:
   뉴욕 → JFK
   로스앤젤레스 → LAX
   시카고 → ORD
\n📍 오세아니아:
   시드니 → SYD
   멜버른 → MEL
\n📊 총 40개 도시 지원

🎉 Phase 2: 확장성 개선 완료!
💡 이제 이런 기능들을 사용할 수 있습니다:
   - add_new_city('제주도', 'CJU')  # 새 도시 추가
   - show_supported_cities()        # 지원 도시 목록
   - validate_city('방콕')          # 도시 유효성 검사

✅ 그룹 1 완료: 모든 함수가 정의되었습니다!
🔧 핵심 함수명 변경 완료:
 

In [ ]:
################도시 코드 관린 수정하여 테스트 해보아야 함. 내일 수 오전에 진행 할것 #################

# =============================================================================
# 🚀 그룹 1: 통일된 함수명 - 마이리얼트립 크롤링 시스템 (리팩토링 완료)
# - 도시 정보를 UNIFIED_CITY_INFO로 통합하여 단일 소스로 관리
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    "LONGER_DELAYS": True,           # 새로 추가
    "MEMORY_CLEANUP_INTERVAL": 5,    # 새로 추가
    "MAX_PRODUCTS_PER_CITY": 2,     # 2 → 10개로 증가⭐⭐⭐⭐⭐⭐⭐⭐⭐
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["바르셀로나"]

# =============================================================================
# 📍 [최종 수정본] 단일 정보 소스 및 리팩토링된 함수
# =============================================================================

# 모든 도시의 상세 정보가 포함된 딕셔너리 (최종 확장본)
UNIFIED_CITY_INFO = {
    # 동남아시아
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK"},
    "아유타야": {"대륙": "아시아", "국가": "태국", "코드": "BKK"}, # 방콕 공항 사용
    "치앙마이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"},
    "빠이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"}, # 치앙마이 공항 사용
    "치앙라이": {"대륙": "아시아", "국가": "태국", "코드": "CEI"},
    "푸켓": {"대륙": "아시아", "국가": "태국", "코드": "HKT"},
    "피피": {"대륙": "아시아", "국가": "태국", "코드": "KBV"}, # 크라비 공항 사용
    "크라비": {"대륙": "아시아", "국가": "태국", "코드": "KBV"},
    "후아힌": {"대륙": "아시아", "국가": "태국", "코드": "HHQ"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG"},
    "쿠알라룸푸르": {"대륙": "아시아", "국가": "말레이시아", "코드": "KUL"},
    "코타키나발루": {"대륙": "아시아", "국가": "말레이시아", "코드": "BKI"},
    "페낭": {"대륙": "아시아", "국가": "말레이시아", "코드": "PEN"},
    "랑카위": {"대륙": "아시아", "국가": "말레이시아", "코드": "LGK"},
    "세부": {"대륙": "아시아", "국가": "필리핀", "코드": "CEB"},
    "보홀": {"대륙": "아시아", "국가": "필리핀", "코드": "TAG"},
    "마닐라": {"대륙": "아시아", "국가": "필리핀", "코드": "MNL"},
    "보라카이": {"대륙": "아시아", "국가": "필리핀", "코드": "MPH"}, # 카티클란 공항
    "다낭": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"},
    "호이안": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"}, # 다낭 공항 사용
    "후에": {"대륙": "아시아", "국가": "베트남", "코드": "HUI"},
    "호치민": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"},
    "무이네": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"}, # 호치민 공항 사용
    "푸꾸옥": {"대륙": "아시아", "국가": "베트남", "코드": "PQC"},
    "나트랑": {"대륙": "아시아", "국가": "베트남", "코드": "CXR"},
    "하노이": {"대륙": "아시아", "국가": "베트남", "코드": "HAN"},
    "달랏": {"대륙": "아시아", "국가": "베트남", "코드": "DLI"},
    "발리": {"대륙": "아시아", "국가": "인도네시아", "코드": "DPS"},
    "프놈펜": {"대륙": "아시아", "국가": "캄보디아", "코드": "PNH"},
    "시엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"},
    "씨엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, # 시엠립 동의어
    "비엔티안": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"},
    "방비엥": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"}, # 비엔티안 공항 사용
    "루앙프라방": {"대륙": "아시아", "국가": "라오스", "코드": "LPQ"},

    # 동북아시아
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX"},
    "교토": {"대륙": "아시아", "국가": "일본", "코드": "KIX"}, # 오사카 공항 사용
    "나고야": {"대륙": "아시아", "국가": "일본", "코드": "NGO"},
    "후쿠오카": {"대륙": "아시아", "국가": "일본", "코드": "FUK"},
    "벳푸": {"대륙": "아시아", "국가": "일본", "코드": "OIT"}, # 오이타 공항 사용
    "오이타": {"대륙": "아시아", "국가": "일본", "코드": "OIT"},
    "구마모토": {"대륙": "아시아", "국가": "일본", "코드": "KMJ"},
    "오키나와": {"대륙": "아시아", "국가": "일본", "코드": "OKA"},
    "미야코지마": {"대륙": "아시아", "국가": "일본", "코드": "MMY"},
    "삿포로": {"대륙": "아시아", "국가": "일본", "코드": "CTS"},
    "타이베이": {"대륙": "아시아", "국가": "대만", "코드": "TPE"},
    "상하이": {"대륙": "아시아", "국가": "중국", "코드": "PVG"},
    "베이징": {"대륙": "아시아", "국가": "중국", "코드": "PEK"},
    "하이난(싼야)": {"대륙": "아시아", "국가": "중국", "코드": "SYX"},
    "마카오": {"대륙": "아시아", "국가": "마카오", "코드": "MFM"},
    
    # 남아시아
    "뉴델리": {"대륙": "아시아", "국가": "인도", "코드": "DEL"},
    "뭄바이": {"대륙": "아시아", "국가": "인도", "코드": "BOM"},
    "카트만두": {"대륙": "아시아", "국가": "네팔", "코드": "KTM"},
    "포카라": {"대륙": "아시아", "국가": "네팔", "코드": "PKR"},

    # 한국
    "서울": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "부산": {"대륙": "아시아", "국가": "대한민국", "코드": "PUS"},
    "제주": {"대륙": "아시아", "국가": "대한민국", "코드": "CJU"},
    "대구": {"대륙": "아시아", "국가": "대한민국", "코드": "TAE"},
    "광주": {"대륙": "아시아", "국가": "대한민국", "코드": "KWJ"},
    "여수": {"대륙": "아시아", "국가": "대한민국", "코드": "RSU"},
    "인천": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "김포": {"대륙": "아시아", "국가": "대한민국", "코드": "GMP"},

    # 유럽
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG"},
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR"},
    "더블린": {"대륙": "유럽", "국가": "아일랜드", "코드": "DUB"},
    "로마": {"대륙": "유럽", "국가": "이탈리아", "코드": "FCO"},
    "피렌체": {"대륙": "유럽", "국가": "이탈리아", "코드": "FLR"},
    "베네치아": {"대륙": "유럽", "국가": "이탈리아", "코드": "VCE"},
    "밀라노": {"대륙": "유럽", "국가": "이탈리아", "코드": "MXP"},
    "바르셀로나": {"대륙": "유럽", "국가": "스페인", "코드": "BCN"},
    "마드리드": {"대륙": "유럽", "국가": "스페인", "코드": "MAD"},
    "세비야": {"대륙": "유럽", "국가": "스페인", "코드": "SVQ"},
    "그라나다": {"대륙": "유럽", "국가": "스페인", "코드": "GRX"},
    "이비자": {"대륙": "유럽", "국가": "스페인", "코드": "IBZ"},
    "리스본": {"대륙": "유럽", "국가": "포르투갈", "코드": "LIS"},
    "포르투": {"대륙": "유럽", "국가": "포르투갈", "코드": "OPO"},
    "프라하": {"대륙": "유럽", "국가": "체코", "코드": "PRG"},
    "비엔나": {"대륙": "유럽", "국가": "오스트리아", "코드": "VIE"},
    "취리히": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"},
    "인터라켄": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"}, # 취리히 공항 사용
    "암스테르담": {"대륙": "유럽", "국가": "네덜란드", "코드": "AMS"},
    "브뤼셀": {"대륙": "유럽", "국가": "벨기에", "코드": "BRU"},
    "뮌헨": {"대륙": "유럽", "국가": "독일", "코드": "MUC"},
    "베를린": {"대륙": "유럽", "국가": "독일", "코드": "BER"},
    "프랑크푸르트": {"대륙": "유럽", "국가": "독일", "코드": "FRA"},
    "부다페스트": {"대륙": "유럽", "국가": "헝가리", "코드": "BUD"},
    "바르샤바": {"대륙": "유럽", "국가": "폴란드", "코드": "WAW"},
    "크라쿠프": {"대륙": "유럽", "국가": "폴란드", "코드": "KRK"},
    "아테네": {"대륙": "유럽", "국가": "그리스", "코드": "ATH"},
    "산토리니": {"대륙": "유럽", "국가": "그리스", "코드": "JTR"},
    "자그레브": {"대륙": "유럽", "국가": "크로아티아", "코드": "ZAG"},
    "두브로브니크": {"대륙": "유럽", "국가": "크로아티아", "코드": "DBV"},
    "코펜하겐": {"대륙": "유럽", "국가": "덴마크", "코드": "CPH"},
    "스톡홀름": {"대륙": "유럽", "국가": "스웨덴", "코드": "ARN"},
    "오슬로": {"대륙": "유럽", "국가": "노르웨이", "코드": "OSL"},
    "헬싱키": {"대륙": "유럽", "국가": "핀란드", "코드": "HEL"},
    "이스탄불": {"대륙": "유럽", "국가": "터키", "코드": "IST"},

    # 북미
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국", "코드": "LAX"},
    "시카고": {"대륙": "북미", "국가": "미국", "코드": "ORD"},
    "하와이": {"대륙": "북미", "국가": "미국", "코드": "HNL"},
    "샌프란시스코": {"대륙": "북미", "국가": "미국", "코드": "SFO"},
    "라스베이거스": {"대륙": "북미", "국가": "미국", "코드": "LAS"},
    "워싱턴 D.C.": {"대륙": "북미", "국가": "미국", "코드": "IAD"},
    "보스턴": {"대륙": "북미", "국가": "미국", "코드": "BOS"},
    "시애틀": {"대륙": "북미", "국가": "미국", "코드": "SEA"},
    "밴쿠버": {"대륙": "북미", "국가": "캐나다", "코드": "YVR"},
    "토론토": {"대륙": "북미", "국가": "캐나다", "코드": "YYZ"},
    "칸쿤": {"대륙": "북미", "국가": "멕시코", "코드": "CUN"},

    # 오세아니아
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD"},
    "멜버른": {"대륙": "오세아니아", "국가": "호주", "코드": "MEL"},
    "괌": {"대륙": "오세아니아", "국가": "괌", "코드": "GUM"},
    "사이판": {"대륙": "오세아니아", "국가": "북마리아나 제도", "코드": "SPN"},
    "오클랜드": {"대륙": "오세아니아", "국가": "뉴질랜드", "코드": "AKL"},
    
    # 중동
    "두바이": {"대륙": "아시아", "국가": "아랍에미리트", "코드": "DXB"},
}



print(f"✅ UNIFIED_CITY_INFO 로드 완료! {len(UNIFIED_CITY_INFO)}개 도시 지원")

# =============================================================================
# 🔧 핵심 함수들 - 단일 정보 소스(UNIFIED_CITY_INFO) 사용
# =============================================================================

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (UNIFIED_CITY_INFO 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        code = info.get("코드", city_name[:3].upper())
        return code
    return city_name[:3].upper()

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (사전 정의된 값만 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 정의되지 않은 도시에 대한 기본값
        return "기타", "기타"

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (기존: get_product_name_by_type → 새로운: get_product_name)"""
    print(f"  📊 {url_type} 상품명 수집 중...")
    
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """✅ 가격 수집 (기존: get_price_fixed → 새로운: get_price)"""
    print(f"  💰 가격 정보 수집 중...")
    
    price_selectors = [
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 87, 87)']"),
        (By.CSS_SELECTOR, "span[style*='color: red']"),
        (By.CSS_SELECTOR, ".price-discount"),
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        (By.XPATH, "//span[contains(text(), '원') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div/div[2]/span[2]")
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_price = price_element.text.strip()
            
            if any(keyword in found_price for keyword in ['쿠폰', '받기', '다운']):
                continue
                
            if '원' in found_price and any(char.isdigit() for char in found_price):
                return found_price
                
        except TimeoutException:
            continue
    
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (기존: extract_clean_price → 새로운: clean_price) (공용 함수)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    price_pattern = r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?'
    match = re.search(price_pattern, price_text)
    
    if match:
        return match.group(1) + "원"
    else:
        return price_text

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text

def download_image(driver, product_name, city_name, product_index):
    """✅ 이미지 다운로드 (기존: download_image_improved_fixed → 새로운: download_image)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': '',
            'path': '',
            'size': 0
        }
        
    print(f"  🖼️ 대표 상품 이미지 다운로드 중...")
    
    image_selectors = [
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.XPATH, "//img[contains(@alt, '상품')]"),
        (By.CSS_SELECTOR, "img[src*='cdn']"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            
            for img_element in img_elements:
                img_url = img_element.get_attribute('src')
                if img_url and img_url.startswith('http'):
                    try:
                        width = img_element.get_attribute('width') or img_element.get_attribute('naturalWidth')
                        height = img_element.get_attribute('height') or img_element.get_attribute('naturalHeight')
                        
                        if width and height:
                            if int(width) < 100 or int(height) < 100:
                                continue
                    except:
                        pass
                    
                    break
            
            if img_url:
                break
                
        except Exception:
            continue

    if img_url:
        try:
            city_code = get_city_code(city_name)
            img_filename = f"{city_code}_{product_index:03d}.jpg"
            
            img_folder = "myrealtripthumb_img"
            os.makedirs(img_folder, exist_ok=True)
            
            img_path = os.path.join(img_folder, img_filename)
            
            urllib.request.urlretrieve(img_url, img_path)
            
            file_size = os.path.getsize(img_path)
            
            if file_size < 1024:
                os.remove(img_path)
                return {
                    'status': '다운로드 실패 (파일 너무 작음)',
                    'filename': img_filename,
                    'path': '',
                    'size': 0
                }
            
            print(f"  ✅ 이미지 다운로드 완료! ({file_size:,} bytes)")
            return {
                'status': '다운로드 완료',
                'filename': img_filename,
                'path': img_path,
                'size': file_size
            }
            
        except Exception as e:
            print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}")
            return {
                'status': f'다운로드 실패: {type(e).__name__}',
                'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
                'path': '',
                'size': 0
            }
    else:
        return {
            'status': '이미지 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'size': 0
        }

def save_results(products_data):
    """✅ 데이터 저장 (기존: save_myrealtrip_data → 새로운: save_results)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    df = pd.DataFrame(products_data)
    csv_path = f"myrealtrip_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📁 개별 CSV 저장 완료: {csv_path}")
    
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "city": city_name
        }
    }
    
    try:
        with open('data_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        print(f"📁 메타데이터 저장 완료: data_metadata.json")
    except Exception as e:
        print(f"⚠️ 메타데이터 저장 실패: {e}")
    
    return csv_path

# =============================================================================
# 🚀 Phase 2: 확장성 개선 시스템 (리팩토링된 버전)
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장 (UNIFIED_CITY_INFO 기반)"""
    enhanced_city_data = {
        "version": "3.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(UNIFIED_CITY_INFO)
    }

    for city_name, info in UNIFIED_CITY_INFO.items():
        enhanced_city_data["cities"][city_name] = {
            "code": info.get("코드", "N/A"),
            "continent": info.get("대륙", "기타"),
            "country": info.get("국가", "기타")
        }
    
    try:
        with open('city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ city_codes.json 파일 생성 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드 (UNIFIED_CITY_INFO와 동기화)"""
    if not os.path.exists('city_codes.json'):
        print("📝 city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
    
    try:
        with open('city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        loaded_cities = city_data.get("cities", {})
        
        for city, info in loaded_cities.items():
            if city not in UNIFIED_CITY_INFO:
                 UNIFIED_CITY_INFO[city] = {
                     "대륙": info.get("continent"),
                     "국가": info.get("country"),
                     "코드": info.get("code")
                 }
        
        print(f"✅ city_codes.json 로드 및 동기화 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 코드의 UNIFIED_CITY_INFO를 사용합니다.")

def add_new_city(city_name, airport_code, continent="기타", country="기타", update_file=True):
    """새로운 도시를 UNIFIED_CITY_INFO에 추가하는 함수"""
    global UNIFIED_CITY_INFO
    
    if city_name not in UNIFIED_CITY_INFO:
        UNIFIED_CITY_INFO[city_name] = {
            "대륙": continent,
            "국가": country,
            "코드": airport_code
        }
        print(f"✅ 메모리에 추가: {city_name} → {airport_code} ({continent}, {country})")
        if update_file:
            create_city_codes_file()
        return True
    else:
        print(f"이미 존재하는 도시입니다: {city_name}")
        return False

def show_supported_cities():
    """지원하는 도시 목록 표시 (UNIFIED_CITY_INFO 기반)"""
    print("\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    cities_by_continent = {}
    for city, info in UNIFIED_CITY_INFO.items():
        continent = info.get("대륙", "기타")
        if continent not in cities_by_continent:
            cities_by_continent[continent] = []
        cities_by_continent[continent].append(city)

    for continent, cities in sorted(cities_by_continent.items()):
        print(f"\n📍 {continent}:")
        for city in sorted(cities):
            code = UNIFIED_CITY_INFO[city].get("코드", "N/A")
            print(f"   {city} → {code}")
    
    print(f"\n📊 총 {len(UNIFIED_CITY_INFO)}개 도시 지원")
    print("="*50)

def validate_city(city_name):
    """도시명 유효성 검사 (UNIFIED_CITY_INFO 기반)"""
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in UNIFIED_CITY_INFO:
        code = UNIFIED_CITY_INFO[city_name].get("코드", "N/A")
        return True, f"지원하는 도시입니다. ({code})"
    
    similar_cities = [c for c in UNIFIED_CITY_INFO if city_name.lower() in c.lower() or c.lower() in city_name.lower()]
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    global CONFIG
    
    scalability_config = {
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def initialize_file_system():
    """파일 시스템 초기화 및 설정 (리팩토링된 버전)"""
    print("🔧 Phase 2: 확장성 개선 시스템 초기화...")
    
    update_config_for_scalability()
    
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        load_city_codes_from_file()
    
    print("✅ Phase 2 시스템 초기화 완료!")
    return True

# =============================================================================
# 🛠️ 기존 유틸리티 함수들 (Phase 2 시스템과 함께 유지)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭 (안정성 강화)"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            driver.execute_script("arguments[0].click();", view_all_button)
            
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
        
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print("  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def get_rating(driver):
    """평점 정보 수집 (기존 함수 유지)"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집"""
    print(f"  🌐 언어 정보 찾는 중...")
    language_selectors = [
        (By.XPATH, "//span[contains(text(), '언어')]"),
        (By.XPATH, "//span[contains(text(), '한국어')]"),
        (By.XPATH, "//span[contains(text(), '영어')]"),
        (By.XPATH, "//span[contains(text(), 'Korean')]"),
        (By.XPATH, "//span[contains(text(), 'English')]"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            language_text = language_element.text.strip()
            
            language_keywords = ['언어', '한국어', '영어', '중국어', '일본어', 'Korean', 'English']
            
            if any(keyword in language_text for keyword in language_keywords):
                print(f"  ✅ 언어 정보 발견: {language_text}")
                return language_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

# =============================================================================
# Phase 2 시스템 자동 실행
# =============================================================================

try:
    initialize_file_system()
    
    # quick_add_cities()는 모든 도시가 UNIFIED_CITY_INFO에 포함되어 더 이상 필요 없음
    
    show_supported_cities()
    
    print("\n🎉 Phase 2: 확장성 개선 완료!")
    print("💡 이제 이런 기능들을 사용할 수 있습니다:")
    print("   - add_new_city('제주도', 'CJU', '아시아', '대한민국')  # 새 도시 추가")
    print("   - show_supported_cities()        # 지원 도시 목록")
    print("   - validate_city('방콕')          # 도시 유효성 검사")
    
except Exception as e:
    print(f"❌ Phase 2 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

print("\n" + "="*60)
print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print("="*60)
print("🔧 핵심 함수명 변경 완료:")
print("   get_product_name_by_type() → get_product_name()")
print("   get_price_fixed() → get_price()")
print("   download_image_improved_fixed() → download_image()")
print("   extract_clean_price() → clean_price()")
print("   extract_clean_rating() → clean_rating()")
print("   save_myrealtrip_data() → save_results()")
print("="*60)
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2,3,4에서 함수 호출을 통일된 함수명으로 변경하세요!")
print("🚨 Gemini 지적사항 모두 해결 완료! 안전하게 실행 가능합니다!")


🔧 Selenium 버전: 4.25.0
✅ UNIFIED_CITY_INFO 로드 완료! 110개 도시 지원
🔧 Phase 2: 확장성 개선 시스템 초기화...
⚙️ CONFIG 확장성 설정 업데이트 완료!
✅ city_codes.json 로드 및 동기화 완료! (110개 도시)
📅 마지막 업데이트: 2025-07-16 00:11:21
✅ Phase 2 시스템 초기화 완료!

🌍 지원하는 도시 목록:

📍 북미:
   뉴욕 → JFK
   라스베이거스 → LAS
   로스앤젤레스 → LAX
   밴쿠버 → YVR
   보스턴 → BOS
   샌프란시스코 → SFO
   시애틀 → SEA
   시카고 → ORD
   워싱턴 D.C. → IAD
   칸쿤 → CUN
   토론토 → YYZ
   하와이 → HNL

📍 아시아:
   광주 → KWJ
   교토 → KIX
   구마모토 → KMJ
   김포 → GMP
   나고야 → NGO
   나트랑 → CXR
   다낭 → DAD
   달랏 → DLI
   대구 → TAE
   델리 → DEL
   도쿄 → NRT
   두바이 → DXB
   랑카위 → LGK
   루앙프라방 → LPQ
   마닐라 → MNL
   마카오 → MFM
   미야코지마 → MMY
   발리 → DPS
   방콕 → BKK
   베이징 → PEK
   벳푸 → OIT
   보라카이 → MPH
   보홀 → TAG
   부산 → PUS
   비엔티안 → VTE
   빠이 → CNX
   삿포로 → CTS
   상하이 → PVG
   서울 → ICN
   세부 → CEB
   시엠립 → REP
   싱가포르 → SIN
   아유타야 → BKK
   여수 → RSU
   오사카 → KIX
   오이타 → OIT
   오키나와 → OKA
   인천 → ICN
   제주 → CJU
   치앙라이 → CEI
   치앙마이 → CNX
   카트만두 → KTM
   코타키나발루 → BKI
   쿠알라룸푸르 → KUL
   크라비 → KBV
   타이베이

In [35]:
# 🚀 그룹 2: 도시 하드코딩 해결 - 드라이버 실행 + 도시 검색
# (이 셀을 실행하면 브라우저가 열리고 그룹 1에서 설정한 도시 검색까지 완료됩니다)

print("🔄 마이리얼트립 크롤링 시작!")

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 크롬 드라이버 실행
try:
    driver = setup_driver()
except Exception as e:
    print(f"❌ 드라이버 설정 실패: {e}")
    print("⚠️ 그룹 1을 먼저 실행했는지 확인하세요!")

# 썸네일 폴더 생성
if CONFIG["SAVE_IMAGES"]:
    img_folder_path = os.path.abspath("") + "/myrealtripthumb_img"
    try:
        shutil.rmtree(img_folder_path)
    except:
        pass
    os.makedirs(img_folder_path, exist_ok=True)
    print("📁 이미지 폴더 생성 완료")

# 🆕 그룹 1 설정에서 도시 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

print("="*60)
print(f"🌏 그룹 1 설정 도시: {city_name} 검색 시작!")
print(f"⚙️  설정: 재시도 {CONFIG['RETRY_COUNT']}회, 타임아웃 {CONFIG['WAIT_TIMEOUT']}초")
print("="*60)

print(f"🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)")
print(f"📍 현재 작업: {city_name} - 진행중")
print("-" * 50)
print(f"  🌍 대륙: {continent} | 국가: {country}")

try:
    # 1. 메인 페이지로 이동
    print("  📱 마이리얼트립 메인 페이지 이동 중...")
    retry_operation(
        lambda: go_to_main_page(driver), 
        "메인 페이지 이동"
    )
    print(f"  ✅ 마이리얼트립 페이지 열기 완료")
    
    # 2. 검색창 찾기 및 입력 (🆕 동적 도시명 사용)
    retry_operation(
        lambda: find_and_fill_search(driver, city_name), 
        f"{city_name} 검색창 입력"
    )

    # 3. 검색 버튼 클릭
    retry_operation(
        lambda: click_search_button(driver), 
        "검색 버튼 클릭"
    )

    # 4. 팝업 처리 (선택사항)
    try:
        handle_popup(driver)
    except Exception as e:
        print(f"  ℹ️ 팝업 처리 중 오류 (무시됨): {type(e).__name__}")

    # 5. 전체 상품 보기 클릭 (선택사항)
    try:
        click_view_all(driver)
    except Exception as e:
        print(f"  ℹ️ 전체 상품 보기 처리 중 오류 (무시됨): {type(e).__name__}")

    print(f"\n✅ 그룹 2 완료: {city_name} 검색 성공!")
    print(f"🎯 현재 상태: {city_name} 상품 목록 페이지에 있습니다")
    print("🚀 다음: 그룹 3을 실행하여 URL 수집 테스트하세요!")
    
except TimeoutException as e:
    print(f"  ⏰ {city_name}: 페이지 로딩 시간 초과")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 시간 초과")
    
except NoSuchElementException as e:
    print(f"  🔍 {city_name}: 필요한 웹 요소를 찾을 수 없음")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 요소 없음")
    
except Exception as e:
    print(f"  ❌ {city_name}: 예상치 못한 오류 - {type(e).__name__}: {e}")
    print("❌ 그룹 2 실패: 예상치 못한 오류")
    print("💡 그룹 1을 먼저 실행했는지 확인하세요!")
    print(f"💡 CITIES_TO_SEARCH에 '{city_name}'이 올바르게 설정되었는지 확인하세요!")
    

🔄 마이리얼트립 크롤링 시작!
🔄 결과 저장소 초기화 완료
✅ 크롬 드라이버 실행 성공!
Windows
📁 이미지 폴더 생성 완료
🌏 그룹 1 설정 도시: 바르셀로나 검색 시작!
⚙️  설정: 재시도 3회, 타임아웃 10초
🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)
📍 현재 작업: 바르셀로나 - 진행중
--------------------------------------------------
  🌍 대륙: 유럽 | 국가: 스페인
  📱 마이리얼트립 메인 페이지 이동 중...
  ✅ 마이리얼트립 페이지 열기 완료
  🔍 '바르셀로나' 검색창 찾는 중...
  ✅ 검색창을 찾았습니다!
  📝 '바르셀로나' 키워드 입력 완료
  🔎 검색 버튼 찾는 중...
  ✅ 검색 버튼 클릭 성공!
  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.
  📋 전체 상품 보기 버튼 찾는 중...
  ✅ 전체 상품 보기 클릭 성공!

✅ 그룹 2 완료: 바르셀로나 검색 성공!
🎯 현재 상태: 바르셀로나 상품 목록 페이지에 있습니다
🚀 다음: 그룹 3을 실행하여 URL 수집 테스트하세요!


In [36]:
# 🔍 그룹 3: URL 수집 테스트 (24개 완전 수집 버전)
# (이 셀을 실행하면 현재 페이지에서 Products + Offers 24개 URL을 모두 수집합니다)

print("🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)")
print("="*50)

def collect_all_24_urls(driver):
    print(f"  🔍 Products와 Offers URL 모두 수집 중...")
    
    # 페이지 로딩 완료 대기
    time.sleep(5)
    
    all_links = driver.find_elements(By.TAG_NAME, "a")
    print(f"  📊 전체 링크 수: {len(all_links)}개")
    
    products_urls = []
    offers_urls = []
    
    for link in all_links:
        try:
            href = link.get_attribute('href')
            if href:
                if 'experiences.myrealtrip.com/products/' in href:
                    if href not in products_urls:
                        products_urls.append(href)
                elif 'myrealtrip.com/offers/' in href:
                    if href not in offers_urls:
                        offers_urls.append(href)
        except:
            continue
    
    # 두 리스트 합치기 (products 먼저, offers 나중에)
    all_urls = products_urls + offers_urls
    
    print(f"  ✅ Products URL: {len(products_urls)}개")
    print(f"  ✅ Offers URL: {len(offers_urls)}개")
    print(f"  🎉 총 수집: {len(all_urls)}개")
    
    return all_urls

try:
    # 24개 URL 수집 실행
    print("📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...")
    
    collected_urls = collect_all_24_urls(driver)
    
    if collected_urls:
        print(f"\n🎉 URL 수집 성공!")
        print(f"📈 총 {len(collected_urls)}개의 상품/오퍼 URL을 찾았습니다!")
        
        # 설정된 개수와 비교
        max_products = CONFIG['MAX_PRODUCTS_PER_CITY']
        will_crawl = min(len(collected_urls), max_products)
        
        print(f"🔢 설정된 크롤링 개수: {max_products}개")
        print(f"🎯 실제 크롤링할 개수: {will_crawl}개")
        
        print(f"\n📋 수집된 URL 목록:")
        print("-" * 80)
        for i, url in enumerate(collected_urls, 1):
            url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
            print(f"  {i:2d}. {url_type}: {url}")
        
        # URL 유효성 체크
        print(f"\n🔍 URL 유효성 체크:")
        products_count = sum(1 for url in collected_urls if '/products/' in url)
        offers_count = sum(1 for url in collected_urls if '/offers/' in url)
        
        print(f"  ✅ Products: {products_count}개")
        print(f"  ✅ Offers: {offers_count}개")
        print(f"  🎯 총 유효 URL: {products_count + offers_count}개")
        
        if len(collected_urls) >= 20:
            print("\n🚀 URL 수집 대성공! 다음 단계로 진행 가능합니다!")
            print("✅ 그룹 3 완료: 24개 URL 수집 테스트 성공")
            print("🎯 다음: 그룹 4를 실행하여 24개 상품/오퍼 크롤링을 시작하세요!")
        else:
            print("\n⚠️ 수집된 URL이 예상보다 적습니다.")
            
    else:
        print("❌ URL 수집 실패: 상품/오퍼 URL을 찾을 수 없습니다")
        print("💡 추가 디버깅:")
        
        # 디버깅 정보
        try:
            page_source = driver.page_source
            products_count = page_source.count('experiences.myrealtrip.com/products/')
            offers_count = page_source.count('myrealtrip.com/offers/')
            print(f"   📄 페이지 소스에서 products 패턴: {products_count}회")
            print(f"   📄 페이지 소스에서 offers 패턴: {offers_count}회")
            
        except Exception as debug_error:
            print(f"   ❌ 디버깅 실패: {debug_error}")
        
        print("🔄 그룹 2를 다시 실행해보세요!")
        
except NameError as e:
    print(f"❌ 변수 오류: {e}")
    print("💡 그룹 1과 그룹 2를 먼저 실행했는지 확인하세요!")
    
except Exception as e:
    print(f"❌ URL 수집 중 오류: {type(e).__name__}: {e}")
    print("💡 가능한 해결책:")
    print("   1. 그룹 2가 성공적으로 완료되었는지 확인")
    print("   2. 브라우저가 방콕 상품 목록 페이지에 있는지 확인")
    print("   3. 인터넷 연결 상태 확인")

print("\n" + "="*50)

🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)
📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...
  🔍 Products와 Offers URL 모두 수집 중...
  📊 전체 링크 수: 52개
  ✅ Products URL: 22개
  ✅ Offers URL: 2개
  🎉 총 수집: 24개

🎉 URL 수집 성공!
📈 총 24개의 상품/오퍼 URL을 찾았습니다!
🔢 설정된 크롤링 개수: 2개
🎯 실제 크롤링할 개수: 2개

📋 수집된 URL 목록:
--------------------------------------------------------------------------------
   1. 🛍️ Product: https://experiences.myrealtrip.com/products/3410190
   2. 🛍️ Product: https://experiences.myrealtrip.com/products/3430505
   3. 🛍️ Product: https://experiences.myrealtrip.com/products/3517343
   4. 🛍️ Product: https://experiences.myrealtrip.com/products/3430539
   5. 🛍️ Product: https://experiences.myrealtrip.com/products/3417847
   6. 🛍️ Product: https://experiences.myrealtrip.com/products/3410191
   7. 🛍️ Product: https://experiences.myrealtrip.com/products/3417843
   8. 🛍️ Product: https://experiences.myrealtrip.com/products/3860315
   9. 🛍️ Product: https://experiences.myrealtrip.com/products/3516980
  10. 🛍️ Produc

In [37]:
# 🚀 하이브리드 구조 적용 - 수정된 그룹 4
# (그룹 1에서 모든 라이브러리 import 완료: re, json, datetime 포함)

# 도시 코드는 그룹 1에서 정의된 CITY_CODES 사용 (전 세계 도시 포함)


def get_city_code(city_name):
    """도시명으로 공항 코드 반환"""
    code = CITY_CODES.get(city_name, city_name[:3].upper())
    print(f"  🏙️ {city_name} → {code}")
    return code

def extract_clean_price(price_text):
    """가격 텍스트에서 실제 숫자 추출"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    price_patterns = [
        r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?',  # "15,000원~" 형태
        r'(\d+)\s*원[~-]?',                 # "15000원~" 형태  
        r'(\d{1,3}(?:,\d{3})*)',           # 숫자만 (,포함)
    ]
    
    for pattern in price_patterns:
        match = re.search(pattern, price_text)
        if match:
            clean_price = match.group(1) + "원"
            print(f"    💰 가격 추출: '{price_text[:30]}...' → '{clean_price}'")
            return clean_price
    
    if any(keyword in price_text for keyword in ['할인', '쿠폰', '특가', '이벤트']):
        print(f"    🎫 할인정보: {price_text[:50]}...")
        return f"할인정보: {price_text}"
    
    print(f"    ⚠️ 가격 패턴 없음: {price_text[:30]}...")
    return price_text

def extract_clean_rating(rating_text):
    """평점 텍스트에서 숫자만 추출"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        clean_rating = match.group(1)
        print(f"    ⭐ 평점 추출: '{rating_text}' → '{clean_rating}'")
        return clean_rating
    
    return rating_text

def create_hierarchical_image_path(continent, country, city, filename):
    """대륙-국가-도시 계층 구조로 이미지 경로 생성"""
    # 마이리얼트립 전용 기본 경로 (현재 폴더 기준)
    base_folder = "myrealtripthumb_img"  # myrealtrip/ 제거
    
    # 계층 구조 경로 생성
    hierarchical_path = os.path.join(base_folder, continent, country, city)
    
    # 전체 파일 경로
    full_path = os.path.join(hierarchical_path, filename)
    
    print(f"  📁 계층 경로: {hierarchical_path}")
    print(f"  📷 파일 경로: {full_path}")
    
    return hierarchical_path, full_path

def create_image_filename(city_name, product_index):
    """공항 코드 + 순서 기반 이미지 파일명 생성"""
    city_code = get_city_code(city_name)
    filename = f"{city_code}_{product_index:03d}.jpg"
    print(f"  📷 이미지 파일명: {filename}")
    return filename

def get_price_fixed(driver):
    """마이리얼트립 실제 가격 수집 (수정된 버전)"""
    print(f"  💰 가격 정보 수집 중...")
    
    price_selectors = [
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 71, 71)']"),
        (By.CSS_SELECTOR, "span[style*='color:#ff4747']"),
        (By.CSS_SELECTOR, ".text-red"),
        (By.XPATH, "//span[contains(text(), '원~') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.XPATH, "//span[contains(text(), '원-') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.XPATH, "//span[contains(text(), ',') and contains(text(), '원') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_elements = driver.find_elements(selector_type, selector_value)
            
            for element in price_elements:
                try:
                    price_text = element.text.strip()
                    
                    if (price_text and 
                        '원' in price_text and 
                        any(char.isdigit() for char in price_text) and
                        not any(keyword in price_text for keyword in ['쿠폰', '할인', '받기', '다운', '적립', '포인트', '혜택', '클릭'])):
                        
                        print(f"  ✅ 가격 발견: {price_text}")
                        return price_text
                        
                except Exception:
                    continue
                    
        except Exception:
            continue
    
    print(f"  ⚠️ 가격 정보를 찾을 수 없습니다")
    return "정보 없음"

def get_main_product_image_url(driver):
    """대표 상품 이미지 URL 찾기 (수정된 버전)"""
    print(f"  🖼️ 대표 이미지 URL 찾는 중...")
    
    image_selectors = [
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".product-main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.CSS_SELECTOR, ".image-gallery img:first-child"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        (By.XPATH, "//img[@width and @height and (@width > '300' or @height > '300')]"),
        (By.CSS_SELECTOR, "img[src*='cdn'][src*='large']"),
        (By.CSS_SELECTOR, "img[src*='cdn'][src*='main']"),
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, "img[src*='http']"),
    ]

    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            
            for img_element in img_elements:
                try:
                    img_url = img_element.get_attribute('src')
                    
                    if img_url and img_url.startswith('http'):
                        try:
                            width = img_element.get_attribute('width') or img_element.size.get('width', 0)
                            height = img_element.get_attribute('height') or img_element.size.get('height', 0)
                            
                            if width and height and (int(str(width).replace('px','')) < 100 or int(str(height).replace('px','')) < 100):
                                continue
                                
                        except:
                            pass
                        
                        exclude_patterns = ['logo', 'icon', 'banner', 'ad', 'avatar', 'profile']
                        if any(pattern in img_url.lower() for pattern in exclude_patterns):
                            continue
                        
                        print(f"  ✅ 대표 이미지 URL 발견: {img_url[:60]}...")
                        return img_url
                        
                except Exception:
                    continue
                    
        except Exception:
            continue
    
    print(f"  ⚠️ 대표 이미지 URL을 찾을 수 없습니다")
    return None

def download_image_hybrid(driver, product_name, continent, country, city_name, product_index):
    """하이브리드 구조용 이미지 다운로드"""
    
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': None,
            'filepath': None,
            'filesize': 0
        }
    
    print(f"  🖼️ 하이브리드 구조로 이미지 다운로드 시작...")
    
    img_url = get_main_product_image_url(driver)
    
    if not img_url:
        return {
            'status': '이미지 URL 없음',
            'filename': None,
            'filepath': None,
            'filesize': 0
        }

    try:
        # 파일명 생성
        filename = create_image_filename(city_name, product_index)
        
        # 하이브리드 계층 구조 경로 생성
        hierarchical_path, full_path = create_hierarchical_image_path(
            continent, country, city_name, filename
        )
        
        # 폴더 생성 (계층 구조)
        os.makedirs(hierarchical_path, exist_ok=True)
        print(f"  ✅ 계층 폴더 생성 완료: {hierarchical_path}")
        
        # 이미지 다운로드
        import urllib.request
        urllib.request.urlretrieve(img_url, full_path)
        
        # 파일 검증
        if os.path.exists(full_path):
            file_size = os.path.getsize(full_path)
            if file_size > 1000:  # 1KB 이상이어야 유효한 이미지
                print(f"  ✅ 하이브리드 구조 이미지 저장 완료! (크기: {file_size:,} bytes)")
                return {
                    'status': '다운로드 완료',
                    'filename': filename,
                    'filepath': full_path,
                    'relative_path': f"{continent}/{country}/{city_name}/{filename}",
                    'filesize': file_size,
                    'original_url': img_url
                }
            else:
                print(f"  ❌ 이미지 파일이 너무 작습니다 ({file_size} bytes)")
                os.remove(full_path)
                return {
                    'status': '파일 너무 작음',
                    'filename': None,
                    'filepath': None,
                    'filesize': 0
                }
        else:
            print(f"  ❌ 이미지 파일 생성 실패")
            return {
                'status': '파일 생성 실패',
                'filename': None,
                'filepath': None,
                'filesize': 0
            }
            
    except Exception as e:
        print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
        return {
            'status': f'다운로드 실패: {type(e).__name__}',
            'filename': None,
            'filepath': None,
            'filesize': 0
        }

def save_myrealtrip_data(products_data):
    """마이리얼트립 데이터를 하이브리드 구조로 저장"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    # 현재 폴더 사용 (이미 myrealtrip 폴더 안이므로)
    mrt_folder = "."  # 현재 디렉토리
    
    # 사이트 정보 추가
    for product in products_data:
        product['사이트'] = "마이리얼트립"
        product['사이트_코드'] = "MRT"
        product['업데이트_시간'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # 개별 CSV 저장 (현재 폴더에)
    df = pd.DataFrame(products_data)
    csv_path = "myrealtrip_products.csv"  # 현재 폴더에 직접 저장
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"✅ 마이리얼트립 개별 CSV 저장: {csv_path}")
    print(f"📊 저장된 상품 수: {len(products_data)}개")
    
    # 메타데이터 업데이트
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "image_path": "myrealtripthumb_img"  # 경로 수정
        }
    }
    
    with open("data_metadata.json", 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 메타데이터 업데이트 완료: data_metadata.json")
    
    return csv_path

# === 메인 크롤링 코드 (하이브리드 구조) ===

print("🚀 하이브리드 구조 그룹 4: 도시별 크롤링!")
print("="*80)

# 🆕 그룹 1 설정에서 도시 가져오기  
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

urls_to_crawl = collected_urls[:CONFIG['MAX_PRODUCTS_PER_CITY']]
total_products = len(urls_to_crawl)

print(f"📊 그룹 1 설정값: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품")
print(f"🎯 실제 크롤링 대상: {total_products}개 상품")
print(f"🌍 위치: {continent} > {country} > {city_name}")
print(f"✈️ 공항 코드: {get_city_code(city_name)}")
print(f"📁 이미지 저장: myrealtripthumb_img/{continent}/{country}/{city_name}/")

# 크롤링할 URL 목록 출력
print("\n📋 크롤링할 상품 목록:")
for i, url in enumerate(urls_to_crawl, 1):
    url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
    image_name = create_image_filename(city_name, i)
    print(f"  {i}. {url_type}: {url}")
    print(f"     → 이미지: {continent}/{country}/{city_name}/{image_name}")

print("="*80)

# 결과 저장 리스트
all_results = []

# 각 상품별 크롤링
for product_index, product_url in enumerate(urls_to_crawl, 1):
    print_product_progress(product_index, total_products, f"상품 {product_index}")
    
    # 기본값 설정
    product_name = "정보 없음"
    price_raw = "정보 없음"
    price_clean = "정보 없음"
    rating_raw = "정보 없음"
    rating_clean = "정보 없음"
    review_count = ""
    language = ""
    img_info = {'status': '처리 안됨', 'filename': None, 'filepath': None}
    url_type = "Product" if "/products/" in product_url else "Offer"
    
    try:
        # 🔗 개별 상품 페이지로 이동
        print(f"    🔗 상품 {product_index} URL로 이동 중...")
        print(f"    📍 {url_type}: {product_url}")
        
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
        
        if driver.current_url != product_url:
            print(f"    ⚠️ URL 변경됨: {driver.current_url}")
        else:
            print(f"    ✅ 정상 이동 완료")
        
        # 상품 정보 수집
        try:
            product_name = get_product_name(driver)
            print(f"    ✅ 상품명 수집 완료")
        except Exception as e:
            print(f"    ⚠️ 상품명 수집 실패: {type(e).__name__}")
            product_name = f"수집실패_{product_index}"

        try:
            price_raw = get_price_fixed(driver)
            price_clean = extract_clean_price(price_raw)
            print(f"    ✅ 가격 수집 완료")
        except Exception as e:
            print(f"    ⚠️ 가격 수집 실패: {type(e).__name__}")

        try:
            rating_raw = get_rating(driver)
            rating_clean = extract_clean_rating(rating_raw)
            print(f"    ✅ 평점 수집 완료")
        except Exception as e:
            print(f"    ⚠️ 평점 수집 실패: {type(e).__name__}")

        try:
            review_count = get_review_count(driver)
            if review_count:
                print(f"    ✅ 리뷰수 수집: {review_count}")
        except Exception as e:
            print(f"    ⚠️ 리뷰수 수집 실패: {type(e).__name__}")

        try:
            language = get_language(driver)
            if language:
                print(f"    ✅ 언어 수집: {language}")
        except Exception as e:
            print(f"    ⚠️ 언어 수집 실패: {type(e).__name__}")

        try:
            img_info = download_image_hybrid(driver, product_name, continent, country, city_name, product_index)
        except Exception as e:
            print(f"    ⚠️ 이미지 처리 실패: {type(e).__name__}")

        # 결과 저장 (하이브리드 구조)
        result = {
            '번호': len(all_results) + 1,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': get_city_code(city_name),
            '상품번호': product_index,
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            '언어': language,
            '이미지_파일명': img_info.get('filename', ''),
            '이미지_상대경로': img_info.get('relative_path', ''),
            '이미지_전체경로': img_info.get('filepath', ''),
            '이미지_상태': img_info.get('status', ''),
            '이미지_크기': img_info.get('filesize', 0),
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }
        
        all_results.append(result)

        # 상품 정보 요약 출력
        safe_name = str(product_name)[:40] + "..." if len(str(product_name)) > 40 else str(product_name)
        print(f"    🎉 상품 {product_index} 완료!")
        print(f"       타입: {url_type}")
        print(f"       상품명: {safe_name}")
        print(f"       가격: {price_clean}")
        print(f"       평점: {rating_clean}")
        print(f"       이미지: {img_info.get('relative_path', '없음')}")
        if review_count:
            print(f"       리뷰수: {review_count}")
        if language:
            print(f"       언어: {language}")
        
        # 다음 상품을 위한 휴식
        if product_index < total_products:
            wait_time = random.uniform(2, 4)
            print(f"    ⏰ 다음 상품까지 {wait_time:.1f}초 대기...")
            time.sleep(wait_time)

    except Exception as e:
        print(f"    ❌ 상품 {product_index} 오류: {type(e).__name__}: {e}")
        # 오류 시 기본 정보 저장
        result = {
            '번호': len(all_results) + 1,
            '대륙': continent,
            '국가': country, 
            '도시': city_name,
            '공항코드': get_city_code(city_name),
            '상품번호': product_index,
            '상품타입': url_type,
            '상품명': f"오류_{product_index}",
            '가격_원본': "수집실패",
            '가격_정제': "수집실패",
            '평점_원본': "수집실패",
            '평점_정제': "수집실패", 
            '리뷰수': "",
            '언어': "",
            '이미지_파일명': "",
            '이미지_상대경로': "",
            '이미지_전체경로': "",
            '이미지_상태': "처리실패",
            '이미지_크기': 0,
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': f'오류({type(e).__name__})'
        }
        all_results.append(result)
        continue

# 최종 결과 처리
print(f"\n🎉 하이브리드 구조 크롤링 완료!")
print(f"📊 수집 결과: {len(all_results)}개 상품")

# 타입별 통계
product_count = len([r for r in all_results if r['상품타입'] == 'Product'])
offer_count = len([r for r in all_results if r['상품타입'] == 'Offer']) 
success_count = len([r for r in all_results if r['상태'] == '완전수집'])
image_success = len([r for r in all_results if r['이미지_상태'] == '다운로드 완료'])

print(f"🛍️ Products: {product_count}개")
print(f"🏷️ Offers: {offer_count}개")
print(f"✅ 크롤링 성공: {success_count}개")
print(f"📷 이미지 성공: {image_success}개")
print(f"📊 전체 성공률: {success_count/len(all_results)*100:.1f}%")
print(f"🖼️ 이미지 성공률: {image_success/len(all_results)*100:.1f}%")

# 상품별 간단 요약
print(f"\n📋 수집된 상품 요약:")
for i, result in enumerate(all_results, 1):
    status_emoji = "✅" if result['상태'] == '완전수집' else "❌"
    img_emoji = "📷" if result['이미지_상태'] == '다운로드 완료' else "🚫"
    print(f"  {i}. {status_emoji}{img_emoji} [{result['상품타입']}] {result['상품명'][:30]}...")
    print(f"      가격: {result['가격_정제']} | 평점: {result['평점_정제']}")
    if result['이미지_상대경로']:
        print(f"      이미지: {result['이미지_상대경로']}")

# 하이브리드 구조로 저장
if all_results:
    try:
        saved_path = save_myrealtrip_data(all_results)
        
        print(f"\n🏆 하이브리드 구조 저장 완료!")
        print(f"📁 개별 CSV: {saved_path}")
        print(f"📁 이미지 폴더: myrealtripthumb_img/{continent}/{country}/{city_name}/")
        print(f"📁 메타데이터: data_metadata.json")
        
        # 폴더 구조 확인
        img_folder = f"myrealtripthumb_img/{continent}/{country}/{city_name}"
        if os.path.exists(img_folder):
            img_files = os.listdir(img_folder)
            print(f"\n📷 저장된 이미지: {len(img_files)}개")
            for img_file in img_files:
                file_size = os.path.getsize(os.path.join(img_folder, img_file))
                print(f"     📸 {img_file} ({file_size:,} bytes)")
        
    except Exception as e:
        print(f"❌ 하이브리드 저장 실패: {e}")

print(f"\n🎯 하이브리드 구조 마이리얼트립 크롤링 완료!")
print(f"🏗️ 구조: 개별 관리 + 계층 이미지 저장")
print(f"🚀 다음: 클룩, KKday도 같은 구조로 확장 가능!")
print(f"💡 통합 분석: ProductDataManager로 전체 사이트 비교 분석!")

🚀 하이브리드 구조 그룹 4: 도시별 크롤링!
📊 그룹 1 설정값: 2개 상품
🎯 실제 크롤링 대상: 2개 상품
🌍 위치: 유럽 > 스페인 > 바르셀로나
  🏙️ 바르셀로나 → BCN
✈️ 공항 코드: BCN
📁 이미지 저장: myrealtripthumb_img/유럽/스페인/바르셀로나/

📋 크롤링할 상품 목록:
  🏙️ 바르셀로나 → BCN
  📷 이미지 파일명: BCN_001.jpg
  1. 🛍️ Product: https://experiences.myrealtrip.com/products/3410190
     → 이미지: 유럽/스페인/바르셀로나/BCN_001.jpg
  🏙️ 바르셀로나 → BCN
  📷 이미지 파일명: BCN_002.jpg
  2. 🛍️ Product: https://experiences.myrealtrip.com/products/3430505
     → 이미지: 유럽/스페인/바르셀로나/BCN_002.jpg
    🎯 상품 진행률: [██████████░░░░░░░░░░] 50.0% (1/2)
    📦 현재 상품: 상품 1
    🔗 상품 1 URL로 이동 중...
    📍 Product: https://experiences.myrealtrip.com/products/3410190
    ✅ 정상 이동 완료
  📊 Product 상품명 수집 중...
    ✅ 상품명 수집 완료
  💰 가격 정보 수집 중...
  ✅ 가격 발견: 25,000원~
    💰 가격 추출: '25,000원~...' → '25,000원'
    ✅ 가격 수집 완료
    ⭐ 평점 추출: '4.9 ·' → '4.9'
    ✅ 평점 수집 완료
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 4,126개
    ✅ 리뷰수 수집: 후기 4,126개
  🌐 언어 정보 찾는 중...
  ✅ 언어 정보 발견: 한국어
    ✅ 언어 수집: 한국어
  🖼️ 하이브리드 구조로 이미지 다운로드 시작...
  🖼️ 대표 이미지 URL 찾는 중...
  

In [ ]:
## 현재 디버깅 해야 됨. url 수집과정에 오루 3번 그룹에서 하면 되는데 여기서 하면서 오류 생김
# # =============================================================================
# 🔍 그룹 2: 통일된 함수명 - 검색 및 URL 수집 시스템 (함수명 통일 완료)
# 그룹 1에서 정의된 통일된 함수명들을 사용
# =============================================================================

print("🔍 그룹 2: 검색 및 URL 수집 시스템 시작!")
print("📋 그룹 1의 통일된 함수명들을 사용합니다:")
print("   - get_product_name()")
print("   - get_price()")
print("   - download_image()")
print("   - clean_price()")
print("   - clean_rating()")
print("   - save_results()")
print("   - get_city_code(), get_city_info() (공용)")

def search_and_collect_urls(driver, city_name):
    """도시별 검색 및 상품 URL 수집 (통일된 함수명 사용)"""
    print(f"\n🔍 {city_name} 도시 검색 시작...")
    
    try:
        # 1. 메인 페이지 이동
        print("📍 1단계: 메인 페이지 이동")
        go_to_main_page(driver)
        
        # 2. 검색창 찾기 및 입력
        print("📍 2단계: 검색 실행")
        find_and_fill_search(driver, city_name)
        
        # 3. 검색 버튼 클릭
        click_search_button(driver)
        
        # 4. 팝업 처리
        print("📍 3단계: 팝업 처리")
        handle_popup(driver)
        
        # 5. 전체 상품 보기 클릭
        print("📍 4단계: 전체 상품 보기")
        click_view_all(driver)
        
        # 6. 상품 URL 수집
        print("📍 5단계: 상품 URL 수집")
        product_urls = collect_page_urls(driver)
        
        # 7. 수집 결과 요약
        print(f"\n✅ {city_name} 검색 완료!")
        print(f"📊 수집된 상품 URL: {len(product_urls)}개")
        
        # 8. 도시 코드 확인 (통일된 함수명 사용)
        city_code = get_city_code(city_name)
        continent, country = get_city_info(city_name)
        
        print(f"🏙️ 도시 정보: {city_name} ({city_code}) - {continent}, {country}")
        
        return product_urls
        
    except Exception as e:
        print(f"❌ {city_name} 검색 실패: {type(e).__name__}: {e}")
        return []

def validate_and_filter_urls(product_urls, city_name):
    """수집된 URL 유효성 검사 및 필터링"""
    print(f"\n🔍 {city_name} URL 유효성 검사 중...")
    
    valid_urls = []
    invalid_urls = []
    
    for i, url in enumerate(product_urls):
        print(f"  📋 URL {i+1}/{len(product_urls)} 검사 중...")
        
        # URL 기본 유효성 검사
        if not url or not url.startswith('http'):
            invalid_urls.append(url)
            continue
            
        # 마이리얼트립 경험 상품 URL 확인
        if '/experiences/' not in url:
            invalid_urls.append(url)
            continue
            
        # 중복 제거
        if url in valid_urls:
            continue
            
        valid_urls.append(url)
    
    print(f"  ✅ 유효한 URL: {len(valid_urls)}개")
    print(f"  ❌ 무효한 URL: {len(invalid_urls)}개")
    
    return valid_urls

def prepare_crawling_data(valid_urls, city_name):
    """크롤링을 위한 데이터 준비"""
    print(f"\n🔧 {city_name} 크롤링 데이터 준비 중...")
    
    # 최대 상품 수 제한
    max_products = CONFIG["MAX_PRODUCTS_PER_CITY"]
    if len(valid_urls) > max_products:
        valid_urls = valid_urls[:max_products]
        print(f"  ⚠️ 상품 수를 {max_products}개로 제한했습니다.")
    
    # 크롤링 준비 데이터 생성
    crawling_data = []
    
    for i, url in enumerate(valid_urls):
        # 도시 코드 생성 (통일된 함수명 사용)
        city_code = get_city_code(city_name)
        continent, country = get_city_info(city_name)
        
        item_data = {
            'index': i + 1,
            'url': url,
            'city_name': city_name,
            'city_code': city_code,
            'continent': continent,
            'country': country,
            'status': 'pending'
        }
        
        crawling_data.append(item_data)
    
    print(f"  ✅ {len(crawling_data)}개 상품 크롤링 데이터 준비 완료!")
    
    return crawling_data

def quick_url_validation(driver, sample_url):
    """샘플 URL로 빠른 유효성 검사"""
    print(f"\n🔍 샘플 URL 유효성 검사 중...")
    
    try:
        # 샘플 URL 접속
        driver.get(sample_url)
        time.sleep(random.uniform(2, 4))
        
        # 기본 요소들 확인 (통일된 함수명 사용)
        try:
            # 상품명 존재 확인
            test_name = get_product_name(driver)
            print(f"  ✅ 상품명 수집 가능: {test_name[:30]}...")
            
            # 가격 정보 확인
            test_price = get_price(driver)
            print(f"  ✅ 가격 정보 수집 가능: {test_price}")
            
            # 가격 정제 테스트 (통일된 함수명 사용)
            clean_price_result = clean_price(test_price)
            print(f"  ✅ 가격 정제 결과: {clean_price_result}")
            
        except Exception as e:
            print(f"  ⚠️ 상품 정보 수집 실패: {type(e).__name__}")
            return False
        
        print(f"  ✅ URL 유효성 검사 통과!")
        return True
        
    except Exception as e:
        print(f"  ❌ URL 접속 실패: {type(e).__name__}: {e}")
        return False

def multi_city_search(driver, cities_list):
    """다중 도시 검색 (통일된 함수명 사용)"""
    print(f"\n🌍 다중 도시 검색 시작! ({len(cities_list)}개 도시)")
    
    all_crawling_data = []
    
    for i, city_name in enumerate(cities_list):
        print(f"\n{'='*60}")
        print(f"🔍 도시 {i+1}/{len(cities_list)}: {city_name}")
        print(f"{'='*60}")
        
        # 도시 유효성 검사 (통일된 함수명 사용)
        is_valid, message = validate_city(city_name)
        if not is_valid:
            print(f"  ⚠️ {message}")
            continue
        
        try:
            # 1. 검색 및 URL 수집
            product_urls = search_and_collect_urls(driver, city_name)
            
            if not product_urls:
                print(f"  ⚠️ {city_name}에서 상품 URL을 찾을 수 없습니다.")
                continue
            
            # 2. URL 유효성 검사
            valid_urls = validate_and_filter_urls(product_urls, city_name)
            
            if not valid_urls:
                print(f"  ⚠️ {city_name}에서 유효한 URL을 찾을 수 없습니다.")
                continue
            
            # 3. 크롤링 데이터 준비
            city_crawling_data = prepare_crawling_data(valid_urls, city_name)
            
            # 4. 샘플 URL 유효성 검사
            if city_crawling_data:
                sample_url = city_crawling_data[0]['url']
                if quick_url_validation(driver, sample_url):
                    all_crawling_data.extend(city_crawling_data)
                    print(f"  ✅ {city_name} 검색 완료! {len(city_crawling_data)}개 상품 준비")
                else:
                    print(f"  ❌ {city_name} URL 유효성 검사 실패")
            
            # 5. 중간 결과 저장 (통일된 함수명 사용)
            if CONFIG.get("SAVE_INTERMEDIATE", False):
                temp_data = [{'도시': city_name, 'URL수': len(valid_urls), '준비완료': len(city_crawling_data)}]
                save_intermediate_results(temp_data, f"{city_name}_search_results")
            
        except Exception as e:
            print(f"  ❌ {city_name} 검색 중 오류 발생: {type(e).__name__}: {e}")
            continue
        
        # 도시 간 대기 시간
        if i < len(cities_list) - 1:
            delay = random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"])
            print(f"  💤 다음 도시 검색 전 대기: {delay:.1f}초")
            time.sleep(delay)
    
    # 최종 결과 요약
    print(f"\n{'='*60}")
    print(f"🎉 다중 도시 검색 완료!")
    print(f"{'='*60}")
    print(f"📊 검색 도시: {len(cities_list)}개")
    print(f"📊 총 수집 상품: {len(all_crawling_data)}개")
    
    # 도시별 상품 수 요약
    city_summary = {}
    for item in all_crawling_data:
        city = item['city_name']
        city_summary[city] = city_summary.get(city, 0) + 1
    
    print(f"\n📋 도시별 상품 수:")
    for city, count in city_summary.items():
        city_code = get_city_code(city)  # 통일된 함수명 사용
        print(f"   {city} ({city_code}): {count}개")
    
    return all_crawling_data

def save_search_results(all_crawling_data):
    """검색 결과를 임시 저장 (통일된 함수명 사용)"""
    if not all_crawling_data:
        print("  ⚠️ 저장할 검색 결과가 없습니다.")
        return None
    
    print(f"\n💾 검색 결과 임시 저장 중...")
    
    # 검색 결과 데이터 변환
    search_results = []
    for item in all_crawling_data:
        result = {
            '순번': item['index'],
            '도시': item['city_name'],
            '도시코드': item['city_code'],
            '대륙': item['continent'],
            '국가': item['country'],
            'URL': item['url'],
            '상태': item['status']
        }
        search_results.append(result)
    
    # 임시 저장 (통일된 함수명의 save_intermediate_results 사용)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    temp_filename = f"search_results_{len(search_results)}개_{timestamp}.csv"
    
    try:
        df = pd.DataFrame(search_results)
        df.to_csv(temp_filename, index=False, encoding='utf-8-sig')
        print(f"  ✅ 검색 결과 저장 완료: {temp_filename}")
        return temp_filename
    except Exception as e:
        print(f"  ⚠️ 검색 결과 저장 실패: {e}")
        return None

def run_group2_search():
    """그룹 2 검색 시스템 실행"""
    print(f"\n🚀 그룹 2: 검색 시스템 실행 시작!")
    print(f"📋 검색 대상 도시: {CITIES_TO_SEARCH}")
    
    # 드라이버 설정
    driver = None
    try:
        # 1. 드라이버 초기화
        driver = setup_driver()
        
        # 2. 다중 도시 검색 실행
        all_crawling_data = multi_city_search(driver, CITIES_TO_SEARCH)
        
        if not all_crawling_data:
            print("❌ 검색 결과가 없습니다.")
            return None
        
        # 3. 검색 결과 저장
        saved_file = save_search_results(all_crawling_data)
        
        # 4. 최종 결과 출력
        print(f"\n🎉 그룹 2 검색 완료!")
        print(f"📊 총 {len(all_crawling_data)}개 상품 URL 수집")
        print(f"📁 저장된 파일: {saved_file}")
        print(f"🎯 다음: 그룹 3 (URL 수집) 또는 그룹 4 (메인 크롤링) 실행")
        
        return all_crawling_data
        
    except Exception as e:
        print(f"❌ 그룹 2 검색 실행 중 오류: {type(e).__name__}: {e}")
        return None
        
    finally:
        # 드라이버 종료
        if driver:
            try:
                driver.quit()
                print("✅ 드라이버 종료 완료")
            except:
                pass

def test_unified_functions():
    """통일된 함수명 테스트"""
    print(f"\n🔧 통일된 함수명 테스트 시작...")
    
    # 도시 코드 테스트
    test_cities = ["후쿠오카", "도쿄", "방콕"]
    
    for city in test_cities:
        print(f"\n📍 {city} 테스트:")
        
        # 통일된 함수명 사용
        city_code = get_city_code(city)
        continent, country = get_city_info(city)
        
        print(f"  🏙️ 도시 코드: {city_code}")
        print(f"  🌍 위치: {continent}, {country}")
        
        # 도시 유효성 검사
        is_valid, message = validate_city(city)
        print(f"  ✅ 유효성: {is_valid} - {message}")
    
    # 가격 정제 테스트
    test_prices = [
        "15,000원",
        "최대 5만원 할인 + 25,000원",
        "3,500원부터",
        "정보 없음"
    ]
    
    print(f"\n💰 가격 정제 테스트:")
    for price in test_prices:
        # 통일된 함수명 사용
        clean_result = clean_price(price)
        print(f"  원본: {price} → 정제: {clean_result}")
    
    # 평점 정제 테스트
    test_ratings = [
        "4.9 · 후기 많음",
        "4.2",
        "평점 없음",
        "5.0 ★★★★★"
    ]
    
    print(f"\n⭐ 평점 정제 테스트:")
    for rating in test_ratings:
        # 통일된 함수명 사용
        clean_result = clean_rating(rating)
        print(f"  원본: {rating} → 정제: {clean_result}")
    
    print(f"\n✅ 통일된 함수명 테스트 완료!")


# =============================================================================
# 그룹 2 실행 옵션
# =============================================================================

print("\n" + "="*60)
print("✅ 그룹 2 완료: 검색 시스템 준비 완료!")
print("="*60)
print("🔧 사용 가능한 함수들:")
print("   - search_and_collect_urls(driver, city_name)")
print("   - multi_city_search(driver, cities_list)")
print("   - run_group2_search()  # 전체 검색 실행")
print("   - test_unified_functions()  # 함수명 테스트")
print("="*60)
print("🎯 실행 방법:")
print("   1. test_unified_functions()      # 함수 테스트")
print("   2. run_group2_search()           # 전체 검색 실행")
print("   3. 또는 그룹 4에서 직접 크롤링 실행")
print("="*60)
print("✅ 모든 함수가 그룹 1의 통일된 함수명을 사용합니다!")
print("   get_product_name(), get_price(), clean_price(), clean_rating(), save_results() 등")



🔍 그룹 2: 검색 및 URL 수집 시스템 시작!
📋 그룹 1의 통일된 함수명들을 사용합니다:
   - get_product_name()
   - get_price()
   - download_image()
   - clean_price()
   - clean_rating()
   - save_results()
   - get_city_code(), get_city_info() (공용)

✅ 그룹 2 완료: 검색 시스템 준비 완료!
🔧 사용 가능한 함수들:
   - search_and_collect_urls(driver, city_name)
   - multi_city_search(driver, cities_list)
   - run_group2_search()  # 전체 검색 실행
   - test_unified_functions()  # 함수명 테스트
🎯 실행 방법:
   1. test_unified_functions()      # 함수 테스트
   2. run_group2_search()           # 전체 검색 실행
   3. 또는 그룹 4에서 직접 크롤링 실행
✅ 모든 함수가 그룹 1의 통일된 함수명을 사용합니다!
   get_product_name(), get_price(), clean_price(), clean_rating(), save_results() 등


In [23]:
run_group2_search()


🚀 그룹 2: 검색 시스템 실행 시작!
📋 검색 대상 도시: ['후쿠오카']
✅ 크롬 드라이버 실행 성공!
Windows

🌍 다중 도시 검색 시작! (1개 도시)

🔍 도시 1/1: 후쿠오카

🔍 후쿠오카 도시 검색 시작...
📍 1단계: 메인 페이지 이동
📍 2단계: 검색 실행
  🔍 '후쿠오카' 검색창 찾는 중...
  ✅ 검색창을 찾았습니다!
  📝 '후쿠오카' 키워드 입력 완료
  🔎 검색 버튼 찾는 중...
  ✅ 검색 버튼 클릭 성공!
📍 3단계: 팝업 처리
  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.
📍 4단계: 전체 상품 보기
  📋 전체 상품 보기 버튼 찾는 중...
  ✅ 전체 상품 보기 클릭 성공!
📍 5단계: 상품 URL 수집
  📊 현재 페이지의 상품 URL들을 수집 중...
  ✅ 1개의 상품 URL을 수집했습니다!

✅ 후쿠오카 검색 완료!
📊 수집된 상품 URL: 1개
  🏙️ 후쿠오카 → FUK
🏙️ 도시 정보: 후쿠오카 (FUK) - 기타, 기타

🔍 후쿠오카 URL 유효성 검사 중...
  📋 URL 1/1 검사 중...
  ✅ 유효한 URL: 1개
  ❌ 무효한 URL: 0개

🔧 후쿠오카 크롤링 데이터 준비 중...
  🏙️ 후쿠오카 → FUK
  ✅ 1개 상품 크롤링 데이터 준비 완료!

🔍 샘플 URL 유효성 검사 중...
  📊 Product 상품명 수집 중...
  ⚠️ 상품 정보 수집 실패: NoSuchElementException
  ❌ 후쿠오카 URL 유효성 검사 실패
  💾 중간 결과 저장: temp_중간저장_후쿠오카_search_results_20250715_230926.csv

🎉 다중 도시 검색 완료!
📊 검색 도시: 1개
📊 총 수집 상품: 0개

📋 도시별 상품 수:
❌ 검색 결과가 없습니다.
✅ 드라이버 종료 완료


In [28]:
pip install geopy


   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]

Note: you may need to restart the kernel to use updated packages.
